<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import numpy as np
import random
import sys
import requests
import os
import datetime

In [10]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)

In [11]:
r.encoding = r.apparent_encoding

data = r.text

In [12]:
data[:100]

'\r\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBo'

In [13]:
data = data.split('\r\n')

In [14]:
toc = [l.strip() for l in data[44:130:2]]

In [15]:
# Skip the Table of Contents
data = data[135:]

In [16]:
toc

['ALL’S WELL THAT ENDS WELL',
 'THE TRAGEDY OF ANTONY AND CLEOPATRA',
 'AS YOU LIKE IT',
 'THE COMEDY OF ERRORS',
 'THE TRAGEDY OF CORIOLANUS',
 'CYMBELINE',
 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
 'THE FIRST PART OF KING HENRY THE FOURTH',
 'THE SECOND PART OF KING HENRY THE FOURTH',
 'THE LIFE OF KING HENRY THE FIFTH',
 'THE FIRST PART OF HENRY THE SIXTH',
 'THE SECOND PART OF KING HENRY THE SIXTH',
 'THE THIRD PART OF KING HENRY THE SIXTH',
 'KING HENRY THE EIGHTH',
 'KING JOHN',
 'THE TRAGEDY OF JULIUS CAESAR',
 'THE TRAGEDY OF KING LEAR',
 'LOVE’S LABOUR’S LOST',
 'THE TRAGEDY OF MACBETH',
 'MEASURE FOR MEASURE',
 'THE MERCHANT OF VENICE',
 'THE MERRY WIVES OF WINDSOR',
 'A MIDSUMMER NIGHT’S DREAM',
 'MUCH ADO ABOUT NOTHING',
 'THE TRAGEDY OF OTHELLO, MOOR OF VENICE',
 'PERICLES, PRINCE OF TYRE',
 'KING RICHARD THE SECOND',
 'KING RICHARD THE THIRD',
 'THE TRAGEDY OF ROMEO AND JULIET',
 'THE TAMING OF THE SHREW',
 'THE TEMPEST',
 'THE LIFE OF TIMON OF ATHENS',
 'THE TRAGEDY 

In [20]:
len(toc)

43

In [85]:
# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

In [86]:
locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

In [87]:
# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})

In [88]:
import pandas as pd
df_toc = pd.DataFrame.from_dict(locations, orient='index')

In [89]:
df_toc

,title,start
0,ALL’S WELL THAT ENDS WELL,2777
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739
2,AS YOU LIKE IT,11841
3,THE COMEDY OF ERRORS,14632
4,THE TRAGEDY OF CORIOLANUS,17833
5,CYMBELINE,27807
6,"THE TRAGEDY OF HAMLET, PRINCE OF DENMARK",27825
7,THE FIRST PART OF KING HENRY THE FOURTH,34512
8,THE SECOND PART OF KING HENRY THE FOURTH,39228
9,THE LIFE OF KING HENRY V,42637


In [90]:
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)

In [93]:
df_toc.loc[42, 'end'] = len(data)

In [94]:
df_toc.tail()

,title,start,end
38,A LOVER’S COMPLAINT,162007,162389.0
39,THE PASSIONATE PILGRIM,162390,162629.0
40,THE PHOENIX AND THE TURTLE,162630,162723.0
41,THE RAPE OF LUCRECE,162724,164942.0
42,VENUS AND ADONIS,164943,166768.0


In [105]:
df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [106]:
df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738.0,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840.0,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631.0,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832.0,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806.0,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [20]:
# divide b/w plays and sonets
sonets = data[:2776]
plays = data[2777:]

In [36]:
data[0]

'THE SONNETS'

In [ ]:
def long_lines(lst_ln):
    clean = []
    
    for ln in lst_ln: 
        
        if len(ln) == 0:
            pass
        else:
            pct = len(ln.strip(' ')) / len(ln)

            if pct >= .5:
                clean.append(ln.lstrip())

    return clean

In [ ]:
# May Not be Needed
#sonets = long_lines(sonets)
#plays = long_lines(plays)

## Word Encoding

This is just a start, and is not complete yet. 

In [ ]:
vocab = list(set("\r\n".join(plays).split()))
words = [line.split() for line in plays]

In [ ]:
len(vocab)

## Character Encoding

Using the technique shown in lecture. 

In [24]:
text = '\r\n'.join(sonets)

chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

print(f"Our corpus contains {len(chars)} unique characters.")

Our corpus contains 73 unique characters.


In [29]:
# Create the Sequence Data

maxlen = 150
step = 1

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 100978


In [30]:
import numpy as np

# Specify x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [31]:
x.shape

(100978, 150, 73)

In [ ]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), dropout=0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1028)              4531424   
_________________________________________________________________
dense (Dense)                (None, 73)                75117     
Total params: 4,606,541
Trainable params: 4,606,541
Non-trainable params: 0
_________________________________________________________________


In [34]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [35]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [37]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model.fit(x, y,
          batch_size=1024,
          validation_split=.2,
          epochs=100,
          callbacks=[print_callback, 
                     #EarlyStopping(min_delta=.02, monitor='val_loss', patience=10),
                     tensorboard_callback])

Train on 80782 samples, validate on 20196 samples
Epoch 1/100
 2048/80782 [..............................] - ETA: 26:02 - loss: 4.0877
----- Generating text after Epoch: 0
----- Generating with seed: "Each changing place with that which goes before,
In sequent toil all forwards do contend.
Nativity once in the main of light,
Crawls to maturity, w"
Each changing place with that which goes before,
In sequent toil all forwards do contend.
Nativity once in the main of light,
Crawls to maturity, wcyoObdsGhahotnlrior soGdon
diirhma
s2flhEA srSosiiGnohr
 cuish
swhothuhgn
s8i5hplhsahbhv
!hr Phodhho ihrlhnzolhivuoiRr rhf3h
izws
olhaaonhyloAwisellollhhh,lnOt5htrzhylsTnh criYvl1ehyTsaagrwsFchDbriBo:isnhlnotAtOr.rooslatthh,o,S dsf 5ol ohbrlt?(rtronEhnilr5ghc,svrimash,b, ogBaalOxSgochhwthoif!Rn do5wqcgnn at3krwn.
sdssiil,olamrr Tlohg,r
yrOhwioe
wiwJtlfia3soofvadAytaytc p,mhiGo oyvf rtn5h
 2048/80782 [..............................] - ETA: 1:09:39 - loss: 4.0877

KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN